In [1]:
import pandas as pd #importa a biblioteca pandas
df = pd.read_csv('D:/DidaticaTech/GSPC.csv') #usa a função .read_csv para abrir o dataset
df.head() #exibe as cinco primeiras linhas do dataset

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


In [2]:
df = df.drop('Date', axis = 1) #retira a coluna Date do dataset

In [3]:
df[-2::] #conferência das últimas duas linhas do dataset

#as duas últimas linhas serão separadas em dois datasets: Hoje.csv e Futuro.csv

,Open,High,Low,Close,Adj Close,Volume
17216,2748.459961,2752.610107,2739.510010,2748.800049,2748.800049,3517790000
17217,2753.250000,2772.389893,2748.459961,2772.350098,2772.350098,3651640000


In [4]:
#a variável amanha irá receber a última linha do dataset
amanha = df[-1::]
amanha.to_csv('D:/DidaticaTech/Futuro.csv', index=False)

In [5]:
#variável base para conter todo o dataset, menos a última linha
base = df.drop(df[-10::].index, axis = 0)
base.to_csv('D:/DidaticaTech/Hoje.csv', index=False)

In [ ]:
#variável/coluna target irá receber o valor do fechamento 'Close' do dia de amanhã (linha 17215), e salva com um índice diferente (1)
#o fechamento do dia 1 será movido para o índice 0, e o fechamento da linha 17216 será salvo na variável target, na linha 17215.
#reseta-se os índices para não dar problema.
base['target'] = base['Close'][1:len(base)].reset_index(drop=True)
base.tail()

In [ ]:
#salvando a última linha 17216 na variável prev
prev = base[-1::].drop('target', axis = 1)
prev

In [ ]:
#retiando a última linha da base de dados e salvando o restante na variável treino
treino = base.drop(base[-1::].index, axis = 0)
treino.tail()

In [ ]:
#alterando o valor de target sempre que uma condição for atendida. 
#sempre que o valor de target for maior que o de Close, target receberá o valor de 1 (teve um aumento).
treino.loc[treino['target'] > treino['Close'], 'target'] = 1
treino.tail()

In [ ]:
#sempre que o valor de target for menor do que o do fechamento (Close), target receberá o valor de 0.
treino.loc[treino['target'] != 1, 'target'] = 0
treino.tail()

In [ ]:
#modelo preditivo para treino
y = treino['target']
x = treino.drop('target', axis = 1)

from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3) #30% teste, 70% treino

from sklearn.ensemble import ExtraTreesClassifier
modelo = ExtraTreesClassifier()
modelo.fit(x_treino, y_treino)

resultado = modelo.score(x_teste, y_teste)
print("Acurácia:", resultado)
#a acurácia em torno de 50% não é boa. esse modelo não poderia ser usado em produção.

In [ ]:
print(resultado)

In [ ]:
prev

In [ ]:
modelo.predict(prev)

In [ ]:
#criou a variável/coluna target na variável prev
prev['target'] = modelo.predict(prev)
prev

In [ ]:
#salvando prev no formato .csv
prev.to_csv('D:/DidaticaTech/Prev.csv', index=False)

In [ ]:
amanha

In [ ]:
base.tail()

In [ ]:
#anexando os dados da variável amanha na base de treino.
base = base.append(amanha, sort=True)
base.tail() #comando que mostra as últimas cinco linhas da base de dados